# Crop Disease

# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pls
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load Images

## Data for Traing / Prepocessing

In [3]:
train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator (
    rescale = 1/255.0
)   # Creating an object, that just hold the data, 1 image. 

train_images = train_data_gen.flow_from_directory(             # flow_from_directory/ is a method : will help to execute / instruct to go following folder 
    'drive/MyDrive/DL-Econic/Crop Disease/AugmentedData',   # why augmented, because dl wants more data   
    batch_size = 32,
    target_size = (228, 228)
)


Found 1082 images belonging to 3 classes.


### Code Narration

- code 5: each folder will be each category & each category will holds its own image
- target_size: I dont know the actual size of image, that why i declared to read the image with 228 by 228 in pixel size / height & width
- color not defined, because whatever the color machine will understand
- code 5-8/ method: Return a things that is generator
- Now the data is fridge, when executes it will call one by one, do not load whole data at a time, applied batch, not keeping all together, that helps for processing
- train_test_gen is object & flow_from_directory is method, method applied on object
- 3 classess means 3 folders

## Data for testing / validation || Preprocessing

In [4]:
valid_data_gen = tf.keras.preprocessing.image.ImageDataGenerator (
    rescale = 1 / 255.0     # 1/255 works little better, any value can be applied, depending dataset can be differet, 1 mean 1 imamge
) 

valid_images = valid_data_gen.flow_from_directory (
    'drive/MyDrive/DL-Econic/Crop Disease/data',
    batch_size = 24,                             # total 120 image, which I want to keep in 5 batch so divided by 24
    target_size = (228, 228)

)

Found 120 images belonging to 3 classes.


## Generting Batch by Next Function

- Its return two: 1 is image: 0 & label of that batch or batch_size when index 1 

In [5]:
next(valid_images)[1]  # Labels/ class Stored when index 1, see everything is ready, all data are in properly one hot encoded vector shape, its save more time

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)

### Code Narration - Next Function

- 1 means when its read image, then not store image only, rather store "Label" as well
- See the array, 3 values means 3 category, where there is 1, its represent the class e.g kon ta kon class belong kore
- Like saved/ stored the labels by "One Hot Encoder" whose index is 1 & Images stored in index 0

In [6]:
next(valid_images)[0]  # Image Stored when index 0

array([[[[1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         ...,
         [0.58431375, 0.54509807, 0.2627451 ],
         [0.5882353 , 0.5647059 , 0.26666668],
         [0.5686275 , 0.5568628 , 0.25490198]],

        [[1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         ...,
         [0.58431375, 0.54509807, 0.2627451 ],
         [0.5882353 , 0.5647059 , 0.26666668],
         [0.5686275 , 0.5568628 , 0.25490198]],

        [[1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         ...,
         [0.58431375, 0.54509807, 0.2627451 ],
         [0.5882353 , 0.5647059 , 0.26666668],
         [0.5686275 , 0.5568628 , 0.25490198]],

        ...,

        [[0.5686275 , 0.64705884, 0.25882354],
         [0.5803922 , 0.65882355, 0.27058825]

# Creating Method / Function

- Creating Method for ploting "Loss" & "Accuracy"
- It will take from history & training
- plot for trainin loss or validation loss / error

In [7]:
def plotLoss(history:pd.DataFrame):  # Providing history dataframe, which will keep training history in neuron network & see that over the time how model learned graphically
  plt.figure(figsize=(18,20))
  plt.plot(history.index.values,
           history['loss'], label='Training Error', color = 'darkorange', linewidth=3)
  plt.plot(history.index.values,
           history['val_loss'], label = 'Validation Error', color = 'lightgreen', linewidth=3)
  
  plt.legend()
  plt.grid(True)
  plt.show()

def plotAccuracy(history:pd.DataFrame):
  plt.figure(figsize=(18,20))
  plt.plot(history.index.values, history['accuracy'],
           label='Training Accuracy', color = 'darkorange', linewidth=3)
  plt.plot(history.index.values, history['val_accuracy'],
           label='Validation Accuracy', color='lightgreen', linewidth=3)
  
  plt.legend()
  plt.grid(True)
  plt.show()
           

# Convolution Neuron Network

## Initialize - CNN

In [8]:
cnn = tf.keras.Sequential([
        # Convolution Feature Extraction                 # First layer is convolutional layer, normalization can also be applied, its optional
        tf.keras.layers.Conv2D(                          # There are also 1D, 2D & 3D available
           filters=32, kernel_size=(3,3),
           strides=(1,1), activation='relu',
           input_shape=(228, 228, 3), padding='same'     # Since its first layer, its dont know enough, so input spape must be defined, so that it can understand
        ), 


        # Dimesionality Reduction                     # After 1st conv, there are so many dimenstion, that must be reduced
        tf.keras.layers.MaxPool2D(                    # Maxpool 2D, becuause our filter/ Cov2D, so its must be 2D
           pool_size=(2,2), strides=(1,1), padding='same'
        ),

        
        # Batch Normalization                        # Batch Normalization works better for image processing
        tf.keras.layers.BatchNormalization(),         # after normalization, still its in image for, so need to covert into 1D array
        tf.keras.layers.Flatten(),                  # Flatting for converting 1D array


        # fulluy Connected Network                            # Connecting Artifician Neuron Network
        tf.keras.layers.Dense(units=128, activation='relu'),
        tf.keras.layers.Dropout(rate=0.5, seed=42),              # regularization: dropout, earlystopping, cotroll randomness, seed 42
        tf.keras.layers.Dense(units=3, activation='softmax')   # 3 for 3 class classfication / for multiclass softmax
])

### Code Narration

- First layer is convolutional layer, normalization can also be applied, its optional
- Normalization, not needed actually, becase at the first when retrieve image, we are defining data generated to rescale the image (1/255), so its alreay normalized
- 3x3, total 32 filter like eye, e.g some filter recognize eye, some for nose, some for ear, some for lip etc
- So, multiple filter used for different purpose
- Why multiple feature ? Ans. Each will understand different features
- Kernel Size: that mean 3x3 matrix, stride 1,1 , move space. First move right with 1 space, end then move down with 1 space, first col wise then row wise on the image
- For each pixel, 1 value will be generated, oi filter ta oi image er porsion er sathe multiply hobe component wise then we do sum of this -
- After summation the value we got, that will be passed through the activation wether the pixel will be activated or not or pass or not as a feature that is activation
- Relu: inially in deep layers, relu works better
- Layers needed to add continuouly when 'Sequencial'
- MaxPool: The output we get from first layer, setar upore dimenstion reduce korar jonno, will do MaxPool
- PoolSize : 2D (2,2) : That is matrix size will move over the "Output Matrix" or output image, Stride, move space by col & row wise
- Stride (1,1) : it could be 2,2 ; 3,3 the more the value, the more the dimentionality reduction with the chance of loosing data also, but it will depend  the image size/ image position
- Sequencial in a sence of layers not data, layers are sequencially connected

#### Batch Normalization

- Batch normalization is a method used to make training of artificial neural networks faster and more stable through normalization of the layers' inputs by re-centering and re-scaling.
- Like there are 32 image, like ekta certain corner er pixel value for each 32 image (with same positional pixel value), it will sum & do average (so that all collection of pixel values stays at a similar range/ come under normal form) & by that value it will apply the normalization
- Pixel pixel/ same positional pixel value/ same korner er for 32 imange e.g 0,1
- Batch Normalization for model optimization
- Like standard/ min, max sclar: Standard scaler, protita dataset k average kore okhan theke min k minus korto & std diye divide korto

## Compilation

In [9]:
cnn.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',  # Categorical crosentropy for calculating loss for multiclass
    metrics = ['accuracy']             # Matrics for evaluate that is accuracy
)

## GPU Activation

- Go Runtime > Change runtime type > hardware > GPU >> run all

In [12]:
tf.config.experimental.list_physical_devices()  # Check the CPU & GPU Status, here I have 1 CPU & 1 GPU

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Fit the Model - with GPU

In [14]:
with tf.device('/device:GPU:0'):  # Creating Session: ask to use GPU devise to perform all the follwing opertion
  cnn.fit(
      train_images,  # Generator
      epochs= 25,
      validation_data = valid_images  # Generator
  )

Epoch 1/25
34/34 [==============================] - 599s 17s/step - loss: 85.9909 - accuracy: 0.5961 - val_loss: 16.7322 - val_accuracy: 0.3750
Epoch 2/25
34/34 [==============================] - 7s 205ms/step - loss: 3.8972 - accuracy: 0.5850 - val_loss: 1.2057 - val_accuracy: 0.4500
Epoch 3/25
34/34 [==============================] - 7s 202ms/step - loss: 1.8080 - accuracy: 0.5397 - val_loss: 1.3926 - val_accuracy: 0.3750
Epoch 4/25
34/34 [==============================] - 7s 202ms/step - loss: 1.1202 - accuracy: 0.5841 - val_loss: 1.1591 - val_accuracy: 0.4833
Epoch 5/25
34/34 [==============================] - 7s 202ms/step - loss: 0.8589 - accuracy: 0.6155 - val_loss: 1.1248 - val_accuracy: 0.4333
Epoch 6/25
34/34 [==============================] - 7s 200ms/step - loss: 0.8225 - accuracy: 0.6128 - val_loss: 1.5181 - val_accuracy: 0.4417
Epoch 7/25
34/34 [==============================] - 7s 201ms/step - loss: 0.6611 - accuracy: 0.6590 - val_loss: 0.7231 - val_accuracy: 0.6833
Epoc

- Accuracy fluctuate becasue each iteration train on different batched, that is the chararistic of Stochastic Approach
- Fluctuation will increase more if take one after one image, that why we dont take one & one, rather applied batch
- Application on all at a time also problem, so batch is better option

# Model Summary - CNN

In [15]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 228, 228, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 228, 228, 32)     0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 228, 228, 32)     128       
 ormalization)                                                   
                                                                 
 flatten (Flatten)           (None, 1663488)           0         
                                                                 
 dense (Dense)               (None, 128)               212926592 
                                                                 
 dropout (Dropout)           (None, 128)               0

In [16]:
hist = pd.DataFrame(cnn.history.history)
hist

,loss,accuracy,val_loss,val_accuracy
0,85.990913,0.596118,16.732220,0.375000
1,3.897228,0.585028,1.205655,0.450000
2,1.807974,0.539741,1.392599,0.375000
3,1.120162,0.584104,1.159144,0.483333
4,0.858871,0.615527,1.124847,0.433333
5,0.822478,0.612754,1.518076,0.441667
6,0.661092,0.658965,0.723090,0.683333
7,0.622743,0.675601,0.580110,0.750000
8,0.590127,0.672828,0.531947,0.766667
9,0.623026,0.673752,0.808594,0.508333
